# Multi-Document Agents

In this guide, you learn towards setting up an agent that can effectively answer different types of questions over a larger set of documents.

These questions include the following

- QA over a specific doc
- QA comparing different docs
- Summaries over a specific odc
- Comparing summaries between different docs

We do this with the following architecture:

- setup a "document agent" over each Document: each doc agent can do QA/summarization within its doc
- setup a top-level agent over this set of document agents. Do tool retrieval and then do CoT over the set of tools to answer a question.

## Setup and Download Data

In this section, we'll define imports and then download Wikipedia articles about different cities. Each article is stored separately.

We load in 18 cities - this is not quite at the level of "hundreds" of documents but its still large enough to warrant some top-level document retrieval!

In [1]:
import json

wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
    "Tokyo",
    "Berlin",
    "Lisbon",
    "Paris",
    "London",
    "Atlanta",
    "Munich",
    "Shanghai",
    "Beijing",
    "Copenhagen",
    "Moscow",
    "Cairo",
    "Karachi",
]

In [2]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

/Users/sajalnarang/PycharmProjects/SECInsights/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [19]:
# Load all wiki documents
# city_docs = {}
# for wiki_title in wiki_titles:
#     city_docs[wiki_title] = SimpleDirectoryReader(
#         input_files=[f"data/{wiki_title}.txt"]
#     ).load_data()

In [2]:
cache = {}

In [30]:
import openai

def llm_call(messages, functions=None):
    
    if functions:
        print(f"Calling LLM with messages: {messages}\nand {len(functions)} functions\n\n")
    else:
        print(f"Calling LLM with messages: {messages}\n\n")
    
    if cache.get(tuple([message.get("content") for message in messages])) is not None:
        print("Using cached response")
        return cache.get(tuple([message.get("content") for message in messages]))
    
    openai.api_base = "https://api.openai.com/v1"
    openai.api_key = ""
    
    if functions is None:
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
        )
    else:
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            functions=functions
    )
    
    cache[tuple([message.get("content") for message in messages])] = result
    return result

In [4]:
local_cache = {}

In [31]:
def local_llm_call(messages, functions=None):
    
    if functions:
        print(f"Calling Local LLM with messages: {messages}\nand {len(functions)} functions\n\n")
    else:
        print(f"Calling Local LLM with messages: {messages}\n\n")
        
    if local_cache.get(tuple([message.get("content") for message in messages])) is not None:
        print("Using cached response")
        return local_cache.get(tuple([message.get("content") for message in messages]))
    
    openai.api_base = "http://localhost:8080/v1"
    openai.api_key = "fake-key"
    
    if functions is None:
        result = openai.ChatCompletion.create(
            model="wizard",
            messages=messages,
            temperature=0,
        )
    else:
        result = openai.ChatCompletion.create(
            model="wizard",
            messages=messages,
            temperature=0,
            functions=functions
        )
    
    local_cache[tuple([message.get("content") for message in messages])] = result
    return result

In [32]:
local_llm_call([{"role":"user", "content":"What is the answer to life?"}])

Calling Local LLM with messages: [{'role': 'user', 'content': 'What is the answer to life?'}]


Using cached response


<OpenAIObject chat.completion id=c92d5d45-2a0f-4f9a-9678-27d28ef522ce at 0x103ec3f40> JSON: {
  "created": 1700456871,
  "object": "chat.completion",
  "id": "c92d5d45-2a0f-4f9a-9678-27d28ef522ce",
  "model": "wizard",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "\nThe answer to life, the universe and everything is 42. This was determined by a supercomputer overlord named Deep Thought in the science fiction book \"The Hitchhiker's Guide to the Galaxy\" by Douglas Adams."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 0,
    "completion_tokens": 0,
    "total_tokens": 0
  }
}

In [7]:
local_llm_call([{"role":"user", "content":"What is the answer to life?"}])

Using cached response


<OpenAIObject chat.completion id=c92d5d45-2a0f-4f9a-9678-27d28ef522ce at 0x103ec3f40> JSON: {
  "created": 1700456871,
  "object": "chat.completion",
  "id": "c92d5d45-2a0f-4f9a-9678-27d28ef522ce",
  "model": "wizard",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "\nThe answer to life, the universe and everything is 42. This was determined by a supercomputer overlord named Deep Thought in the science fiction book \"The Hitchhiker's Guide to the Galaxy\" by Douglas Adams."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 0,
    "completion_tokens": 0,
    "total_tokens": 0
  }
}

In [8]:
def to_openai_function(topic):
    return {
        "name": f"tool_{topic.name.replace(' ', '_')}",
        "description": f"This content contains information about {topic.description}. Use this tool if you want to answer any questions about {topic.description}.",
        "parameters": {
            "type": "object",
            "required": ["attribute"],
            "properties": {
                "attribute": {
                    "type": "string",
                    "description": f"Attribute to query about {topic.description}",
                }
            },
        },
    }

# functions = [to_openai_function(topic) for topic in wiki_titles]

In [41]:
information = {}
for topic in wiki_titles:
    with open(f"data/{topic}.txt", "r") as wiki_file:
        wiki_text = wiki_file.read()
    lines = wiki_text.splitlines()
    heading = "General Information"
    topic_info = {}
    for line in lines:
        if line.startswith("=="):
            heading = line[2:-2]
            continue
        else:
            topic_info[heading] = topic_info.get(heading, "") + line + "\n"
    information[topic] = topic_info

AttributeError: 'str' object has no attribute 'name'

In [42]:
initial_response = llm_call([
    {"role":"system", "content":"You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge."},
    {"role":"user", "content":"Compare the economies of Toronto and Atlanta"}
], functions)

In [53]:
initial_message_history = [
    {"role": "system",
     "content": "You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge."},
    {"role": "user", "content": "Compare the economies of Toronto and Atlanta"}
    # {"role": "user", "content": "Which county is Atlanta in?"}
]
initial_local = local_llm_call(initial_message_history, functions)

Topic(
	name=Toronto
	text=Toronto is the most populous city in Canada and the capital city of the Canadian province of Ontario. With a recorded population of 2,794,356 in 2021
	it is the fourth-most populous city in North America. The city is the anchor of the Golden Horseshoe
	an urban agglomeration of 9,765,188 people (
	as of 2021
) surrounding the western end of Lake Ontario
	while the Greater Toronto Area proper had a 2021 population of 6,712,341. Toronto is an international centre of business
	finance
	arts
	sports and culture
	and is recognized as one of the most multicultural and cosmopolitan cities in the world.Indigenous peoples have travelled through and inhabited the Toronto area
	located on a broad sloping plateau interspersed with rivers
	deep ravines
	and urban forest
	for more than 10,000 years. After the broadly disputed Toronto Purchase
	when the Mississauga surrendered the area to the British Crown
	the British established the town of York in 1793 and later designat

In [73]:
print(initial_local)

{
  "created": 1697590143,
  "object": "chat.completion",
  "id": "672c945f-11d7-40b8-92ea-7a251b4ce446",
  "model": "wizard",
  "choices": [
    {
      "index": 0,
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "arguments": "{\"attribute\":\"economy\"}",
          "function": "tool_Toronto",
          "name": "tool_Toronto"
        }
      }
    }
  ],
  "usage": {
    "prompt_tokens": 0,
    "completion_tokens": 0,
    "total_tokens": 0
  }
}


In [76]:
# def function_call(function_call):
#     if function_call.name.startswith("tool_"):
#         topic = function_call.name[len("tool_"):]
#         attribute = json.loads(function_call.arguments)["attribute"]
#         print(topic)
#         # read from topic file and store the text in wiki_text
#         with open(f"data/{topic}.txt", "r") as wiki_file:
#             wiki_text_local = wiki_file.read()
#         wiki_text_local = wiki_text_local[:1000]
#         message = f"Context information is below.\n---------------------\n{wiki_text_local}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {attribute}\nAnswer: "
#         print(message)
#         return local_llm_call([
#             {"role":"system", "content":f"You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."},
#             {"role":"user", "content":message}
#         ])
#     else:
#         raise ValueError(f"Unknown function {function_call.name}")
# 
# response = initial_local
# message_history = initial_message_history.copy()
# while response.choices[0].finish_reason == "function_call":
#     print(f"Calling function {response.choices[0].message.function_call.name} with arguments {response.choices[0].message.function_call.arguments}")
#     function_response = function_call(response.choices[0].message.function_call)
#     print(function_response)
#     
#     message_history.append(response.choices[0].message)
#     message_history.append({
#         "role": "function",
#         "name": response.choices[0].message.function_call.name,
#         "content": function_response.choices[0].message.content
#     })
#     
#     response = llm_call(message_history, functions)
#     print(response)

Calling function tool_Atlanta with arguments {"attribute":"economy"}
Atlanta
Context information is below.
---------------------
Atlanta ( at-LAN-tə, or  at-LAN-ə) is the capital and most populous city of the U.S. state of Georgia. It is the seat of Fulton County, although a portion of the city extends into neighboring DeKalb County. With a population of 498,715 living within the city limits, it is the eighth most populous city in the Southeast and 38th most populous city in the United States according to the 2020 U.S. census. It is the core of the much larger Atlanta metropolitan area, which is home to nearly 7 million people, making it the eighth-largest metropolitan area in the United States. Situated among the foothills of the Appalachian Mountains at an elevation of just over 1,000 feet (300 m) above sea level, it features unique topography that includes rolling hills, lush greenery, and the most dense urban tree coverage of any major city in the United States.Atlanta was original

In [77]:
print(response)

{
  "id": "chatcmpl-8ApWkcdU0InSUahcqOpsNRKjb1Y90",
  "object": "chat.completion",
  "created": 1697592042,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Both Toronto and Atlanta have strong and diverse economies, but there are some key differences between the two cities.\n\nToronto's economy is driven by industries such as finance, business services, technology, and film production. The city is home to the Toronto Stock Exchange, which is one of the largest stock exchanges in North America. Toronto's economy is also supported by its status as an international center for arts, sports, and culture, attracting tourists and supporting related industries such as hospitality and retail.\n\nOn the other hand, Atlanta's economy is characterized by its strong presence of Fortune 500 companies, including Coca-Cola, Delta Air Lines, Home Depot, UPS, and AT&T. The city serves as the regional, national, a

In [24]:
class Topic:
    def __init__(self, name, description, text, level, subtopics):
        self.name = name.replace(" ", "_")
        self.description = description
        self.text = text
        self.level = level
        self.subtopics = subtopics
    
    
    def __str__(self):
        return f"Topic(name={self.name}, description={self.description}, level={self.level}, subtopics={self.subtopics})"
    
    
    def __repr__(self):
        return str(self)

In [25]:
def create_topic(name, text, level, description_suffix=""):
    heading_prefix = "=" * (level + 2)
    next_heading_prefix = "=" * (level + 3)
    
    subtopics = []
    if heading_prefix in text:
        subtopic_name = "General Information"
        subtopic_text = ""
        for line in text.splitlines():
            if line.startswith(heading_prefix) and not line.startswith(next_heading_prefix):
                if subtopic_text:
                    subtopics.append(create_topic(subtopic_name, subtopic_text, level + 1, " of " + name + description_suffix))
                subtopic_name = line[level + 3:-level - 3]
                subtopic_text = ""
            elif line:
                subtopic_text += line + "\n"
        if subtopic_text:
            subtopics.append(create_topic(subtopic_name, subtopic_text, level + 1))
    
    return Topic(
        name=name,
        description=name + description_suffix,
        text=text,
        level=level,
        subtopics=subtopics   
    )

topics = []
for wiki_title in wiki_titles:
    with open(f"data/{wiki_title}.txt", "r") as wiki_file:
        wiki_text = wiki_file.read()
    topics.append(create_topic(wiki_title, wiki_text, 0))
print(topics)

[Topic(name=Toronto, description=Toronto, level=0, subtopics=[Topic(name=General_Information, description=General Information of Toronto, level=1, subtopics=[]), Topic(name=Toponymy, description=Toponymy of Toronto, level=1, subtopics=[]), Topic(name=History, description=History of Toronto, level=1, subtopics=[Topic(name=Early_history, description=Early history of History of Toronto, level=2, subtopics=[]), Topic(name=19th_century, description=19th century of History of Toronto, level=2, subtopics=[]), Topic(name=20th_century, description=20th century of History of Toronto, level=2, subtopics=[]), Topic(name=21st_century, description=21st century, level=2, subtopics=[])]), Topic(name=Geography, description=Geography of Toronto, level=1, subtopics=[Topic(name=General_Information, description=General Information of Geography of Toronto, level=2, subtopics=[]), Topic(name=Topography, description=Topography of Geography of Toronto, level=2, subtopics=[]), Topic(name=Neighbourhoods_and_form

In [33]:
def llm_call_with_function_resolution(function_arguments, topic):
    messages = [
        {"role":"system", "content":"You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge."},
        {"role":"user", "content":f"Describe the {function_arguments} of {topic.description}"}
    ]
    functions = [to_openai_function(topic) for topic in topic.subtopics]
    response = llm_call(messages, functions)
    if not response.choices[0].finish_reason == "function_call":
        return response
    
    print(f"Calling function {response.choices[0].message.function_call.name} with arguments {response.choices[0].message.function_call.arguments}")
    function_name = response.choices[0].message.function_call.name
    function_arguments = json.loads(response.choices[0].message.function_call.arguments)["attribute"]
    query_topics = topic.subtopics
    current_topic = next(topic for topic in query_topics if topic.name == function_name[len("tool_"):])
    
    if current_topic.subtopics:
        print(f"Subtopics: {current_topic.subtopics}")
        function_response = llm_call_with_function_resolution(function_arguments, current_topic)
    else:
        function_response = local_llm_call([
            {"role":"system", "content":f"You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."},
            {"role":"user", "content":f"Context information is below.\n---------------------\n{current_topic.text}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {function_arguments}\nAnswer: "}
        ])
    
    return function_response
    
# question = "Compare the religious demographics of Toronto and Atlanta"
question = "How different are the airports in Chicago and Berlin?"

query_topics = topics
functions = [to_openai_function(topic) for topic in query_topics]
message_history = [
    {"role": "system",
     "content": "You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge."},
    {"role": "user", "content": question}
]
response = llm_call(message_history, functions)

while response.choices[0].finish_reason == "function_call":
    print(f"Calling function {response.choices[0].message.function_call.name} with arguments {response.choices[0].message.function_call.arguments}")
    function_name = response.choices[0].message.function_call.name
    function_arguments = json.loads(response.choices[0].message.function_call.arguments)["attribute"]
    query_topics = topics
    current_topic = next(topic for topic in query_topics if topic.name == function_name[len("tool_"):])
    
    function_response = llm_call_with_function_resolution(function_arguments, current_topic)
    
    message_history.append(response.choices[0].message)
    message_history.append({
        "role": "function",
        "name": function_name,
        "content": function_response.choices[0].message.content
    })
    
    response = llm_call(message_history, functions)

print("\n\n\n" + response.choices[0].message.content)

Calling LLM with messages: [{'role': 'system', 'content': 'You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge.'}, {'role': 'user', 'content': 'How different are the airports in Chicago and Berlin?'}]
and 18 functions


Using cached response
Calling function tool_Chicago with arguments {
  "attribute": "airports"
}
Calling LLM with messages: [{'role': 'system', 'content': 'You are an agent designed to answer queries about a set of given cities.\nPlease always use the tools provided to answer a question. Do not rely on prior knowledge.'}, {'role': 'user', 'content': 'Describe the airports of Chicago'}]
and 15 functions


Using cached response
Calling function tool_Infrastructure with arguments {
  "attribute": "Airports"
}
Subtopics: [Topic(name=Transportation, description=Transportation of Infrastructure of Chicago, level=2, subtopics=[Topic(name=General_Information, descript